In [1]:
# 축제/공연/행사 API 추출(1) - contentid, 위도/경도, 축제 시작/종료 날짜
import requests  # http request
import json
import pandas as pd

# 생략없이 전체 행 보기
pd.set_option('display.max_rows', None)

# TourAPI 인증키 통하여 json 형식으로 호출 (시군구 구분)
# 강원도 시/군구 총 18개
# 시군구 코드
numlist=[]
for i in range(1,19):
    numlist.append(str(i))

listdata=[]
for num in numlist:
    url = "https://apis.data.go.kr/B551011/KorService1/searchFestival1?serviceKey=L0b9w8P3nD0elJQA%2FcAQC9CQZke7zOVDGkUnLFgZ3D3jSLWG%2F83IksuL29Cg6QMiQQOVl12FCHQyYUn8B%2Fo%2FtA%3D%3D&numOfRows=1&pageNo=1&MobileOS=ETC&MobileApp=AppTest&arrange=A&listYN=Y&eventStartDate=20240901&areaCode=32&sigunguCode="+num+"&_type=json"
    r = requests.get(url)
    data = r.json()
    
    # 해당 기간에 축제가 있는 경우 vs 없는 경우 나눠서 출력
    if data['response']['body']['items']=="": # 행사가 없을 때
        print(f"행사없음 시군구코드: {num}")
        continue
    else:  # 행사가 있을 때
        item = data['response']['body']['items']['item']
        if item: 
            for j in range(len(item)):
                listdata.append(item[j])

# 주소, contentid, 장소명 추출 후 key:value 형태로 변환
newdictdata = []  # 리스트로 초기화

for i in range(len(listdata)):  # 리스트의 길이만큼 반복
    contentid = listdata[i]['contentid']
    title = listdata[i]['title']
    addr = listdata[i]['addr1'] + listdata[i]['addr2']
    mapx = listdata[i]['mapx']
    mapy = listdata[i]['mapy']
    eventstartdate = listdata[i]['eventstartdate']

    eventenddate = listdata[i]['eventenddate']
    
    # 딕셔너리 형태로 추가
    item_dict = {
        'contentid': contentid,
        '축제명': title,
        '주소': addr,
        '경도': mapx,
        '위도': mapy,
        '축제시작일자': eventstartdate,
        '축제종료일자': eventenddate
    }
    # 리스트에 추가
    newdictdata.append(item_dict) 

# 결과 확인 => list
newdictdata  # 리스트 형태로 출력

# list -> json 문자열 변환
json_str = json.dumps(newdictdata,ensure_ascii=False)

# json 문자열 -> df 변환
dfdata=pd.read_json(json_str) 

# 축제시작일자, 축제종료일자 칼럼 타입 변경(int -> str -> datetime)
dfdata['축제시작일자']=dfdata['축제시작일자'].astype('str')
dfdata['축제종료일자']=dfdata['축제종료일자'].astype('str')

dfdata['축제시작일자']=pd.to_datetime(dfdata['축제시작일자'])
dfdata['축제종료일자']=pd.to_datetime(dfdata['축제종료일자'])

# 아래 url의 contentid 작동을 위해 int -> string 
dfdata=dfdata.astype({'contentid':'string'})

# contentid 칼럼 추출
dfdataci=dfdata.iloc[:,0]

# 위도/경도 칼럼 추출
dfdataci_map=dfdata.iloc[:,[3,4]]

# 축제시작일자, 축제종료일자 칼럼 추출
dfdataci_date=dfdata.iloc[:,5:]

# print(dfdataci.head(5))
# print(dfdataci_map.head(5))
# print(dfdataci_date.head(5))

# print(dfdata.info())
# dfdataci
# print(type(dfdataci)) # Series
# print(dfdataci.dtype)
dfdata.head(5)

,contentid,축제명,주소,경도,위도,축제시작일자,축제종료일자
0,3021124,강릉 보현사 산사천년문화재,강원특별자치도 강릉시 성산면 보현길 396,128.769306,37.736626,2024-10-04,2024-10-05
1,421977,고성통일명태축제,강원특별자치도 고성군 거진읍 거진리11리 해변,128.453102,38.441391,2024-10-11,2024-10-13
2,734219,동해 무릉제,강원특별자치도 동해시 덕골길 10 (천곡동),129.102819,37.514972,2024-09-26,2024-09-29
3,678097,삼척정월대보름제,강원특별자치도 삼척시 엑스포로 45 (성남동),129.160131,37.438341,2025-02-14,2025-02-16
4,1718491,설악문화제,강원특별자치도 속초시 엑스포로 75 (조양동),128.582443,38.190892,2024-10-04,2024-10-06


In [2]:
# 축제/공연/행사 추출(2) - 강원도 시군구/관광지 별 장소명, 주소, 개요 정보

# 위 contentid를 받아서 json 형식으로 호출
# contentid를 conid라는 변수로 받음

listdata2=[]
for index, conid in dfdataci.items():
    url = "http://apis.data.go.kr/B551011/KorService1/detailCommon1?serviceKey=2rTA0AsQYrJXYZI8zaF0J2zdy6kUHExuEVXoYPtGJbcUWAkiSpziy2aw%2BVpKwoyE5DFegvPyKwzFTuKcxY16%2Fw%3D%3D&contentId="+conid+"&defaultYN=Y&addrinfoYN=Y&overviewYN=Y&MobileOS=ETC&MobileApp=AppTest&_type=json"
    r = requests.get(url)
    data2 = r.json()   
    item = data2['response']['body']['items']['item']
    
    # 대괄호 제외하고 listdata2에 넣기
    if item: 
        listdata2.append(item[0])

# TourAPI에서 강원도 관광지 별 장소명, 주소, 개요 정보 가져오기
newdictdata2 = [] 
for i in range(len(listdata2)):  
    title = listdata2[i]['title']
    addr = listdata2[i]['addr1'] + listdata2[i]['addr2']
    overview = listdata2[i]['overview']
    
    # 주소에서 시군구 분류 추출
    sigungu = ''
    if '강릉' in addr:
        sigungu = '강릉시'
    elif '고성' in addr:
        sigungu = '고성군'
    elif '동해' in addr:
        sigungu = '동해시'
    elif '삼척' in addr:
        sigungu = '삼척시'
    elif '속초' in addr:
        sigungu = '속초시'
    elif '양구' in addr:
        sigungu = '양구군'
    elif '양양' in addr:
        sigungu = '양양군'
    elif '영월' in addr:
        sigungu = '영월군'
    elif '원주' in addr:
        sigungu = '원주시'
    elif '인제' in addr:
        sigungu = '인제군'
    elif '정선' in addr:
        sigungu = '정선군' 
    elif '철원' in addr:
        sigungu = '철원군'
    elif '춘천' in addr:
        sigungu = '춘천시'
    elif '태백' in addr:
        sigungu = '태백시'
    elif '평창' in addr:
        sigungu = '평창군'
    elif '홍천' in addr:
        sigungu = '홍천군'
    elif '화천' in addr:
        sigungu = '화천군'
    elif '횡성' in addr:
        sigungu = '횡성군'
    
    # 딕셔너리 형태로 추가
    item_dict = {
        '축제명': title,
        '시군구 분류': sigungu,
        '소재지지번주소': addr,
        '설명': overview
    }
    
    # 리스트에 추가
    newdictdata2.append(item_dict) 

# 결과 확인 => list
# newdictdata2

# list -> json 문자열 변환
json_str2 = json.dumps(newdictdata2,ensure_ascii=False)

# json 문자열 -> df 변환
dfdata2=pd.read_json(json_str2) 

# 위도/경도, 축제시작일자/축제종료일자 칼럼 추가
dfdata_final=pd.concat([dfdata2,dfdataci_map,dfdataci_date],axis=1)
dfdata_final

# DB 연동 사전 작업 - list로 변환
all_df2=[]
for i in range(len(dfdata_final)):
    all_df2.append(dfdata_final.loc[i].to_list())
    
all_df2[:5]

[['강릉 보현사 산사천년문화재',
  '강릉시',
  '강원특별자치도 강릉시 성산면 보현길 396',
  '강릉 보현사 산사천년문화재는 천년고찰 보현사의 전통문화유산을 잇는 산신대재, 자장율사/범일국사/낭원대사의 삼대화상 다례재, 산사음악회와 전통차 음다 등이 어우러지는 10월의 강릉 대표 축제이다. 이번 제4회 축제에서는 지난 3년 동안 개최해오던 산사문화제를 보다 발전시켜 미식도시 강릉을 알릴 수 있는 사찰음식과 강릉 향토음식의 향연까지 마련하였다.',
  128.7693064482,
  37.736626283,
  Timestamp('2024-10-04 00:00:00'),
  Timestamp('2024-10-05 00:00:00')],
 ['고성통일명태축제',
  '고성군',
  '강원특별자치도 고성군 거진읍 거진리11리 해변',
  '고성통일명태축제는 액을 막고 복을 주는 명태의 기복신앙을 토대로 ‘행운’이라는 뜻의 Good-Luck(굿럭)을 행사장 곳곳에서 만날 수 있다. 행운을 여는 ‘굿럭쇼’, 명태음식의 모든 것 ‘굿럭푸드’, 명태 공예품과 가공상품을 만나는 ‘굿럭스토어’, 명태의 역사문화적 가치를 전시하는 ‘굿럭컬처’, 명태 경품대잔치 ‘굿럭게임’, 소울 충만 힙플 해변을 느끼는 ‘굿럭 바이브’ 등 ‘굿럭 시티 고성이 드리는 7가지 행운’이 준비되어 있다.',
  128.4531021013,
  38.4413909993,
  Timestamp('2024-10-11 00:00:00'),
  Timestamp('2024-10-13 00:00:00')],
 ['동해 무릉제',
  '동해시',
  '강원특별자치도 동해시 덕골길 10 (천곡동)',
  "강원도 동해시에서 개최되는 무릉제는 동해시민의 화합과 번영을 염원하는 동해시 최대의 한바탕 잔치로서 동해시 제1명승지인 무릉계곡의 명칭을 이용하여 1984년부터 가을이 시작되는 시기에 열리고 있다. 올해 무릉제는 동해시의 아름다운 사계를 담아 축제를 다채롭게 빛내고자 하는 의미로 '색(色)다른 동해,

In [106]:
# DB 연동 - 강원도 시군구 별 관광지 정보 insert
import cx_Oracle as cx
con2=cx.connect("hr","hr","localhost:1521/xe") 
cur2=con2.cursor() 
sql2="insert into festival(place, sigungu, address, overview, longtitude, altitude, startdate, enddate) values(:1,:2,:3,:4,:5,:6,:7,:8)" 

# insert 내용들
for place, sigungu, address, overview, longtitude, altitude, startdate, enddate in all_df2:
    cur2.execute(sql2,(place, sigungu, address, overview, longtitude, altitude, startdate, enddate))

# commit
con2.commit() 

In [105]:
# 축제 달력용 csv 파일로 출력
dfdata_final.to_csv("9월_이후_(강원도)축제정보.csv", index=False, encoding="utf-8-sig")